In [3]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install estnltk==1.7.2
!pip install evaluate
!pip install seqeval
!pip install nervaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      

In [4]:
# Impordid
import os
import numpy as np
import json
import evaluate
from collections import defaultdict
from datasets import Dataset, DatasetDict
from itertools import product
from datetime import datetime
from nervaluate import Evaluator

# Transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, AutoConfig
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
#from transformers import TFAutoModelForTokenClassification, create_optimizer, AdamWeightDecay
#from transformers.keras_callbacks import KerasMetricCallback

#data_processing.py samas kaustas
from data_processing import (
    TAG2IDX, IDX2TAG, ALL_TAGS,
    get_dataset_paths, preprocess, split_to_token_and_tag,
    transform_set, process_all,
    save_split_to_json, save_dataset_to_json, load_split_from_json, load_dataset_from_json,
    load_and_process_dataset, combine_datasetdicts, load_all)

In [5]:
ewt_dataset, edt_dataset, combined_dataset = load_all(from_json=True)
# sama, mis üleval
#ewt_from_json = load_dataset_from_json('ewt')
#edt_from_json = load_dataset_from_json('edt')
#combined_from_json = load_dataset_from_json()

# nullist
#ewt_dataset, edt_dataset, combined_dataset = load_all(from_json=False)
#ewt_dataset = load_and_process_dataset('ewt')
#edt_dataset = load_and_process_dataset('edt')
#combined_dataset = combine_datasetdicts(ewt_dataset, edt_dataset)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT", max_length=128, padding="max_length", truncation="longest_first", do_lower_case=False, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/410k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
# https://huggingface.co/docs/transformers/en/tasks/token_classification

def tokenize_and_align_labels(sent, label_all=True):
  # Sisend: üks lause
  # Väljund: tokenizeri abil sõnestatud lause, sõnastik sisaldab input_ids, token_type_ids, attention_mask ja labels
  # input_ids on lause 'arvulisel' kujul, igale ID-le vastab mingi sõna (teisendamine funktsiooni tokenizer.convert_ids_to_tokens(input_ids) abil)
  # labels on märgendite list, kus märgendid kattuvad tokenizeri abil sõnestatud lausega

  #tokenized_inputs = tokenizer(sent['tokens'], is_split_into_words=True) #, truncation=True, is_split_into_words=True, max_length=128, padding="max_length"
  tokenized_inputs = tokenizer(sent['tokens'], is_split_into_words=True, max_length=128, padding="max_length", truncation="longest_first")
  labels = []
  word_ids = tokenized_inputs.word_ids()
  prev_word = None

  for word in word_ids:
    if word is None:
      label = -100
    elif word != prev_word:
      label = sent['tags'][word]
    else:
      label = sent['tags'][word] if label_all else -100 # kui label_all, siis 'reet', '##pa', '##67' -> B-PER, B-PER, B-PER
      #label = -100 # reetpa67 -> B-Per; 'reet', '##pa', '##67' -> 11, -100, -100 ehk B-Per
    labels.append(label)
    prev_word = word

  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [8]:
def print_aligned_example(dataset, split='dev', i=0):
  example = dataset[split][i]
  print(f'Lause algselt: {example}')

  aligned_example = tokenize_and_align_labels(example, True)
  tokens = tokenizer.convert_ids_to_tokens(aligned_example.input_ids)
  print(f'Lause nö arvulisel kujul: {aligned_example.input_ids}')
  print(f'Sõnestatud: {tokens}')
  aligned_labels = aligned_example['labels']
  print(f'Märgendid arvulisel kujul paigas: {aligned_labels}')
  aligned_labels_text = [ALL_TAGS[label] for label in aligned_labels if label>=0]
  print(f'Märgendid: {aligned_labels_text}')

print_aligned_example(ewt_dataset, 'test', 2)

Lause algselt: {'id': 2, 'tags': [11, 12, 0, 5, 0, 0, 0, 0, 1, 0, 0, 11, 12, 0, 0, 0, 0], 'tokens': ['Jürgen', 'Zopp', 'kaotas', 'Petange’is', '64000', 'euro', 'suuruse', 'auhinnafondiga', 'Challenger-turniiri', 'poolfinaalis', 'prantslasele', 'Paul-Henri', 'Mathieu’le', '6:7', ',', '2:6', '.']}
Lause nö arvulisel kujul: [2, 11969, 26186, 433, 6063, 9765, 16608, 49883, 1, 719, 9154, 921, 973, 5364, 4551, 42241, 48551, 49894, 251, 8290, 18322, 3689, 5992, 4933, 251, 21000, 7440, 204, 13500, 1, 198, 641, 137, 757, 11, 138, 137, 641, 15, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sõnestatud: ['[CLS]', 'Jürgen', 'Zo', '##pp', 'kaotas', 'Pet', '##ang', '##e', '[UNK]', 'is', '64', '##000', 'euro', 'suuruse', 'auhinna', '##fondiga', 'Challenge', '##r', '-', 'turniiri', 'po

In [11]:
# NB! Kasutades DatasetDict ja Dataset on see siin ebavajalik, saab otse dataset.map() kasutada
def tokenize_dataset(dataset_dict):
    # Sisend: andmestik sõnastikuna, mis sisaldab train/dev/test hulkasid
    # Väljund:
    processed_dataset = {}

    for split_name, split_data in dataset_dict.items():
        #examples = [split_data[idx] for idx in range(len(split_data))]
        #print(examples == split_data)
        #tokenized_dataset = list(map(tokenize_and_align_labels, examples))
        tokenized_dataset = list(map(tokenize_and_align_labels, split_data))
        processed_split = {
            'input_ids': [],
            'attention_mask': [],
            'labels': []
        }

        for item in tokenized_dataset:
            processed_split['input_ids'].append(item['input_ids'])
            processed_split['attention_mask'].append(item['attention_mask'])
            processed_split['labels'].append(item['labels'])

        processed_dataset[split_name] = processed_split

    return processed_dataset

In [19]:
tokenized_ewt = ewt_dataset.map(tokenize_and_align_labels)
tokenized_edt = edt_dataset.map(tokenize_and_align_labels)
tokenized_combined = combined_dataset.map(tokenize_and_align_labels)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Map:   0%|          | 0/30045 [00:00<?, ? examples/s]

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

In [10]:
#data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")
data_collator_pt = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [11]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [ALL_TAGS[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [ALL_TAGS[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    metrics = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

    # märgendikaupa tulemused ka
    for tag, values in results.items():
      if isinstance(values, dict):
        metrics[f"{tag} Precision"] = values["precision"]
        metrics[f"{tag} Recall"] = values["recall"]
        metrics[f"{tag} F1"] = values["f1"]
        metrics[f"{tag} Number"] = values["number"]

    return metrics

In [12]:
# tensorflow variant
def finetune_model(tokenized_dataset, epochs=3, batch_size=8):
  num_train_steps = (len(tokenized_dataset["train"]) // batch_size) * epochs
  optimizer, lr_schedule = create_optimizer(
      init_lr=5e-5,
      num_train_steps=num_train_steps,
      weight_decay_rate=0.01,
      num_warmup_steps=0,
      adam_beta1=0.9,
      adam_beta2=0.99,
      adam_epsilon=1e-06
  )

  model = TFAutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", num_labels=len(ALL_TAGS), id2label=IDX2TAG, label2id=TAG2IDX)

  tf_train_set = model.prepare_tf_dataset(
      tokenized_dataset['train'],
      shuffle=True,
      batch_size=batch_size,
      collate_fn=data_collator,
  )
  tf_validation_set = model.prepare_tf_dataset(
      tokenized_dataset['dev'],
      shuffle=False,
      batch_size=batch_size,
      collate_fn=data_collator,
  )

  model.compile(optimizer=optimizer)

  metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
  callbacks = [metric_callback]

  model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=epochs, callbacks=callbacks)

  return model

#combined_model = finetune_model(tokenized_combined)
#Epoch 3/3
#3755/3755 [==============================] - 619s 165ms/step - loss: 0.0727 - val_loss: 0.1575 - precision: 0.6984 - recall: 0.6915 - f1: 0.6949 - accuracy: 0.9617

In [12]:
bert_config = AutoConfig.from_pretrained(
    "tartuNLP/EstBERT",
    id2label = IDX2TAG,
    label2id = TAG2IDX,
    num_labels = len(IDX2TAG),
    output_hidden_states = False,
    output_attentions = False,
)

In [13]:
def finetune_model_pytorch(tokenized_dataset, epochs=3, batch_size=16, lr=5e-5, scheduler='linear', beta1=0.9, beta2=0.99, epsilon=1e-6, decay=0.01, early_stop_patience=2, freeze=False):

  model = AutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", config=bert_config) #num_labels=len(ALL_TAGS), id2label=IDX2TAG, label2id=TAG2IDX
  model.cuda()
  if freeze:
    for param in model.bert.embeddings.parameters():
      param.requires_grad = False
    for layer in model.bert.encoder.layer[:3]:
      for param in layer.parameters():
        param.requires_grad = False

  training_args = TrainingArguments(
      report_to='none',
      output_dir='./results',
      learning_rate=lr,
      lr_scheduler_type=scheduler, # linear/polynomial
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=epochs,
      weight_decay=decay,
      eval_strategy="epoch",
      save_strategy="epoch",
      optim="adamw_torch",
      load_best_model_at_end=True,
      metric_for_best_model='f1',
      adam_beta1=beta1,
      adam_beta2=beta2,
      adam_epsilon=epsilon,
      fp16=True
  )

  early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=early_stop_patience,
        early_stopping_threshold=0.0001
    )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset['train'],
      eval_dataset=tokenized_dataset['dev'],
      processing_class=tokenizer,
      data_collator=data_collator_pt,
      compute_metrics=compute_metrics,
      callbacks=[early_stopping_callback]
  )

  trainer.train()

  return model, trainer

In [15]:
def grid_search_pt(tokenized_dataset=None, param_grid=None):
  results = []
  best_f1 = 0
  best_model = None
  best_params = None

  if param_grid:
    param_combinations = [dict(zip(param_grid.keys(), v))
                         for v in product(*param_grid.values())]

  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

  for i, params in enumerate(param_combinations, 1):
    print(f"\n{i}/{len(param_combinations)}")
    print(f"Params: {params}")

    try:
      model = AutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", num_labels=len(ALL_TAGS), id2label=IDX2TAG, label2id=TAG2IDX)

      training_args = TrainingArguments(
          report_to='none', # kui seda ei ole, siis kasutab weights&biases, vaja api võtit
          output_dir=f'./results_{timestamp}_{i}',
          learning_rate=params['learning_rate'],
          lr_scheduler_type=params['lr_scheduler_type'],
          per_device_train_batch_size=params['batch_size'],
          per_device_eval_batch_size=params['batch_size'],
          num_train_epochs=params['num_train_epochs'],
          weight_decay=params['weight_decay'],
          eval_strategy="epoch",
          save_strategy="epoch",
          optim="adamw_torch",
          load_best_model_at_end=True,
          metric_for_best_model='f1',
          adam_beta1=params['adam_beta1'],
          adam_beta2=params['adam_beta2'],
          adam_epsilon=params['adam_epsilon'],
          fp16=True
      )

      early_stopping_callback = EarlyStoppingCallback(
          early_stopping_patience=2,
          early_stopping_threshold=0.0001
      )

      trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=tokenized_dataset['train'],
          eval_dataset=tokenized_dataset['dev'],
          processing_class=tokenizer,
          data_collator=data_collator_pt,
          compute_metrics=compute_metrics,
          callbacks=[early_stopping_callback]
      )

      train_result = trainer.train()
      eval_result = trainer.evaluate()

      trial_results = {
          'parameters': params,
          'eval_metrics': eval_result,
          'train_metrics': {
              'train_runtime': train_result.metrics['train_runtime'],
              'train_samples_per_second': train_result.metrics['train_samples_per_second']
          }
      }
      results.append(trial_results)

      if eval_result['eval_f1'] > best_f1:
        best_f1 = eval_result['eval_f1']
        best_model = model
        best_params = params

    except Exception as e:
      print(f"Error {i}: {e}")
      continue

  print(f"parim f1: {best_f1}")
  print(f"parameetrid: {best_params}")

  return best_model, best_params, results

In [16]:
param_grid = {
    'learning_rate': [3e-5, 5e-5],
    'batch_size': [16, 32],
    'num_train_epochs': [3],
    'weight_decay': [0.01],
    'adam_beta1': [0.9],
    'adam_beta2': [0.99],
    'adam_epsilon': [1e-6],
    'lr_scheduler_type': ['linear', 'polynomial']
}

best_model, best_params, all_results = grid_search_pt(tokenized_dataset=tokenized_ewt, param_grid=param_grid)


1/8
Params: {'learning_rate': 3e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.01, 'adam_beta1': 0.9, 'adam_beta2': 0.99, 'adam_epsilon': 1e-06, 'lr_scheduler_type': 'linear'}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,No log,0.276649,0.732797,0.728242,0.730512,0.946187,0.450000,0.346154,0.391304,26,0.428571,0.333333,0.375000,9,0.200000,0.093750,0.127660,64,0.926407,0.923741,0.925072,695,0.433604,0.483384,0.457143,331,0.000000,0.000000,0.000000,1
2,0.196300,0.296388,0.745407,0.756661,0.750992,0.950261,0.434783,0.384615,0.408163,26,0.230769,0.333333,0.272727,9,0.137931,0.062500,0.086022,64,0.930935,0.930935,0.930935,695,0.490862,0.567976,0.526611,331,0.000000,0.000000,0.000000,1
3,0.060600,0.314602,0.785518,0.761101,0.773117,0.952721,0.500000,0.346154,0.409091,26,0.230769,0.333333,0.272727,9,0.395833,0.296875,0.339286,64,0.939219,0.933813,0.936508,695,0.551402,0.534743,0.542945,331,0.000000,0.000000,0.000000,1


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,No log,0.281825,0.732291,0.706927,0.719385,0.945341,0.535714,0.576923,0.555556,26,0.187500,0.333333,0.240000,9,0.421053,0.250000,0.313725,64,0.932945,0.920863,0.926865,695,0.382445,0.368580,0.375385,331,0.000000,0.000000,0.000000,1
2,No log,0.286215,0.767442,0.761989,0.764706,0.951568,0.464286,0.500000,0.481481,26,0.230769,0.333333,0.272727,9,0.382979,0.281250,0.324324,64,0.921986,0.935252,0.928571,695,0.535385,0.525680,0.530488,331,0.000000,0.000000,0.000000,1
3,0.139300,0.307705,0.786111,0.753996,0.769719,0.952183,0.428571,0.461538,0.444444,26,0.111111,0.222222,0.148148,9,0.428571,0.281250,0.339623,64,0.944363,0.928058,0.936139,695,0.556634,0.519637,0.537500,331,0.000000,0.000000,0.000000,1


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


parim f1: 0.7769718948322756
parameetrid: {'learning_rate': 5e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.01, 'adam_beta1': 0.9, 'adam_beta2': 0.99, 'adam_epsilon': 1e-06, 'lr_scheduler_type': 'linear'}


In [18]:
#parim f1: 0.7769718948322756
#parameetrid: {'learning_rate': 5e-05, 'batch_size': 16, 'num_train_epochs': 3, 'weight_decay': 0.01, 'adam_beta1': 0.9, 'adam_beta2': 0.99, 'adam_epsilon': 1e-06, 'lr_scheduler_type': 'linear'}
# 20 epohhi ~ 30min
# dev hulk on väike, kõik märgendid pole esindatud
# f1 0.800557
ewt_mudel, trainer_ewt = finetune_model_pytorch(tokenized_ewt,
                                              epochs=20,
                                              batch_size=16,
                                              lr=5e-05,
                                              scheduler='linear',
                                              beta1=0.9,
                                              beta2=0.99,
                                              epsilon=1e-06,
                                              decay=0.01,
                                              early_stop_patience=5,
                                              freeze=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score 

In [20]:
# 15 epohhi ~1h
edt_mudel, trainer_edt = finetune_model_pytorch(tokenized_edt,
                                              epochs=5,
                                              batch_size=16,
                                              lr=5e-5,
                                              scheduler='linear',
                                              beta1=0.9,
                                              beta2=0.99,
                                              epsilon=1e-6,
                                              decay=0.01,
                                              early_stop_patience=2,
                                              freeze=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.086500,0.100373,0.710533,0.698245,0.704335,0.969748,0.313131,0.455882,0.371257,68,0.605839,0.783019,0.683128,424,0.794183,0.558176,0.655586,636,0.000000,0.000000,0.000000,12,0.583929,0.720264,0.644970,454,0.892791,0.890596,0.891692,1627,0.398113,0.362543,0.379496,582,0.000000,0.000000,0.000000,71
2,0.042700,0.122365,0.688071,0.719153,0.703269,0.968796,0.406250,0.382353,0.393939,68,0.532562,0.867925,0.660090,424,0.813187,0.465409,0.592000,636,0.000000,0.000000,0.000000,12,0.543478,0.770925,0.637523,454,0.900606,0.913337,0.906927,1627,0.408805,0.446735,0.426929,582,0.000000,0.000000,0.000000,71
3,0.020300,0.132498,0.713922,0.724058,0.718954,0.971248,0.391304,0.397059,0.394161,68,0.593103,0.811321,0.685259,424,0.780041,0.602201,0.679681,636,0.000000,0.000000,0.000000,12,0.530744,0.722467,0.611940,454,0.926388,0.912723,0.919505,1627,0.419014,0.408935,0.413913,582,0.000000,0.000000,0.000000,71
4,0.010800,0.151280,0.724120,0.738513,0.731246,0.971765,0.414286,0.426471,0.420290,68,0.602190,0.778302,0.679012,424,0.769231,0.613208,0.682415,636,0.000000,0.000000,0.000000,12,0.579422,0.707048,0.636905,454,0.932584,0.918254,0.925364,1627,0.447964,0.510309,0.477108,582,0.000000,0.000000,0.000000,71
5,0.005700,0.164154,0.720551,0.729479,0.724987,0.971426,0.414286,0.426471,0.420290,68,0.598230,0.797170,0.683519,424,0.773737,0.602201,0.677277,636,0.000000,0.000000,0.000000,12,0.525641,0.722467,0.608534,454,0.933083,0.917025,0.924985,1627,0.450704,0.439863,0.445217,582,0.000000,0.000000,0.000000,71


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

In [21]:
combined_mudel, trainer_combined = finetune_model_pytorch(tokenized_combined,
                                              epochs=3,
                                              batch_size=16,
                                              lr=5e-5,
                                              scheduler='linear',
                                              beta1=0.9,
                                              beta2=0.99,
                                              epsilon=1e-6,
                                              decay=0.01,
                                              early_stop_patience=2,
                                              freeze=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.093400,0.139327,0.731343,0.695800,0.713129,0.966341,0.354430,0.411765,0.380952,68,0.593804,0.766667,0.669253,450,0.799043,0.517829,0.628410,645,0.000000,0.000000,0.000000,12,0.481166,0.764479,0.590604,518,0.941176,0.868217,0.903226,2322,0.504202,0.394304,0.442532,913,0.000000,0.000000,0.000000,72
2,0.040800,0.153315,0.737276,0.733000,0.735132,0.968221,0.420290,0.426471,0.423358,68,0.627953,0.708889,0.665971,450,0.723618,0.669767,0.695652,645,0.000000,0.000000,0.000000,12,0.524781,0.694981,0.598007,518,0.930742,0.897071,0.913596,2322,0.506300,0.484118,0.494961,913,0.000000,0.000000,0.000000,72
3,0.019500,0.173116,0.754022,0.740600,0.747251,0.969888,0.428571,0.397059,0.412214,68,0.596315,0.791111,0.680038,450,0.760396,0.595349,0.667826,645,0.000000,0.000000,0.000000,12,0.559701,0.723938,0.631313,518,0.938948,0.907407,0.922908,2322,0.545673,0.497262,0.520344,913,0.000000,0.000000,0.000000,72


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# ennustamiseks vaja kasutada trainer.predict()

# checkpoint_path = "results/checkpoint-341/"
# tok = AutoTokenizer.from_pretrained(checkpoint_path, max_length=128, padding="max_length", truncation="longest_first")
# mdl = AutoModelForTokenClassification.from_pretrained(checkpoint_path)

# training_args = TrainingArguments(
#       report_to='none',
#       output_dir='./results'
# )
# trainer = Trainer(
#           args=training_args,
#           model=mdl,
#           processing_class=tok,
#           data_collator=data_collator_pt,
#           compute_metrics=compute_metrics,
#       )

In [22]:
def get_predictions(dataset, trainer):
  predictions, labels, _ = trainer.predict(dataset)
  predictions = np.argmax(predictions, axis=2)

  true_predictions = [
        [ALL_TAGS[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
  true_labels = [
        [ALL_TAGS[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

  return true_predictions, true_labels

def get_seqeval_results(predictions, actual):
  results = seqeval.compute(predictions=predictions, references=actual)
  return results

def get_nervaluate_results(predictions, actual):
  evaluator = Evaluator(actual, predictions, tags=['EVE', 'GEP', 'LOC', 'MUU', 'ORG', 'PER', 'PROD', 'UNK'], loader="list")
  results, results_by_tag, result_indices, result_indices_by_tag = evaluator.evaluate()

  return results, results_by_tag, result_indices, result_indices_by_tag

def get_results(test_dataset, trainer):
  seqeval_result = get_seqeval_results(*get_predictions(test_dataset, trainer))
  results, results_by_tag, result_indices, result_indices_by_tag = get_nervaluate_results(*get_predictions(test_dataset, trainer))

  return seqeval_result, results, results_by_tag

def print_results(seqeval_result, nereval_result, nereval_by_tag):
  print("Seqeval tulemused")
  for key in seqeval_result:
    print(key, seqeval_result[key])

  print()
  print("Nervaluate tulemused")
  print("Strict", nereval_result['strict'])

  print("precision", nereval_result['strict']['precision'])
  print("recall", nereval_result['strict']['recall'])
  print("f1", nereval_result['strict']['f1'])

  for tag in nereval_by_tag:
    print(tag, nereval_by_tag[tag]['strict'])

In [23]:
#seqeval_result = get_seqeval_results(*get_predictions(tokenized_ewt['test'], trainer))
#results, results_by_tag, result_indices, result_indices_by_tag = get_nervaluate_results(*get_predictions(tokenized_ewt['test'], trainer))
seqeval_edt, nereval_edt, nereval_by_tag_edt = get_results(tokenized_edt['test'], trainer_edt)
seqeval_ewt, nereval_ewt, nereval_by_tag_ewt = get_results(tokenized_ewt['test'], trainer_ewt)
seqeval_combined, nereval_combined, nereval_by_tag_combined = get_results(tokenized_combined['test'], trainer_combined)

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
print("EDT")
print_results(seqeval_edt, nereval_edt, nereval_by_tag_edt)

EDT
Seqeval tulemused
EVE {'precision': 0.5, 'recall': 0.30952380952380953, 'f1': 0.3823529411764706, 'number': 84}
GEP {'precision': 0.6771084337349398, 'recall': 0.7317708333333334, 'f1': 0.7033792240300376, 'number': 384}
LOC {'precision': 0.7644230769230769, 'recall': 0.7210884353741497, 'f1': 0.7421236872812135, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7514204545454546, 'recall': 0.7256515775034293, 'f1': 0.73831123517097, 'number': 729}
PER {'precision': 0.9311246730601569, 'recall': 0.9179200687580575, 'f1': 0.9244752218134602, 'number': 2327}
PROD {'precision': 0.45685997171145687, 'recall': 0.5474576271186441, 'f1': 0.4980724749421742, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.787317498365657
overall_recall 0.7918036379574841
overall_f1 0.7895541958041958
overall_accuracy 0.9760981202003863

Nervaluate tulemused
Strict {'correct': 3613, 'incorrect': 740, 'partial': 0,

In [25]:
print("EWT")
print_results(seqeval_ewt, nereval_ewt, nereval_by_tag_ewt)

EWT
Seqeval tulemused
EVE {'precision': 0.6304347826086957, 'recall': 0.5087719298245614, 'f1': 0.5631067961165049, 'number': 57}
GEP {'precision': 0.6206896551724138, 'recall': 0.6545454545454545, 'f1': 0.6371681415929203, 'number': 55}
LOC {'precision': 0.29411764705882354, 'recall': 0.45454545454545453, 'f1': 0.35714285714285715, 'number': 55}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.5986842105263158, 'recall': 0.3939393939393939, 'f1': 0.47519582245430814, 'number': 231}
PER {'precision': 0.9479277364505845, 'recall': 0.8146118721461187, 'f1': 0.8762278978388999, 'number': 1095}
PROD {'precision': 0.34146341463414637, 'recall': 0.34806629834254144, 'f1': 0.344733242134063, 'number': 362}
overall_precision 0.7153937947494033
overall_recall 0.6463611859838275
overall_f1 0.6791277258566978
overall_accuracy 0.9444158811768066

Nervaluate tulemused
Strict {'correct': 1199, 'incorrect': 349, 'partial': 0, 'missed': 307, 'spurious': 150, 'possible'

In [26]:
print("Combined")
print_results(seqeval_combined, nereval_combined, nereval_by_tag_combined)

Combined
Seqeval tulemused
EVE {'precision': 0.4418604651162791, 'recall': 0.40425531914893614, 'f1': 0.4222222222222222, 'number': 141}
GEP {'precision': 0.6406844106463878, 'recall': 0.7676537585421412, 'f1': 0.6984455958549223, 'number': 439}
LOC {'precision': 0.7233606557377049, 'recall': 0.7116935483870968, 'f1': 0.717479674796748, 'number': 496}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.715625, 'recall': 0.715625, 'f1': 0.715625, 'number': 960}
PER {'precision': 0.9386181369524984, 'recall': 0.8892460549386324, 'f1': 0.9132653061224489, 'number': 3422}
PROD {'precision': 0.49884526558891457, 'recall': 0.453781512605042, 'f1': 0.4752475247524752, 'number': 952}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7903719207857027
overall_recall 0.7648800249298847
overall_f1 0.7774170559822632
overall_accuracy 0.9703752063265256

Nervaluate tulemused
Strict {'correct': 4909, 'incorrect': 966, 'partial': 0, 'missed